<a href="https://colab.research.google.com/github/ddiaz1999/Face_Mask_Detection_System/blob/master/Face_Mask_Detection_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <strong>Face Mask Detection</strong>

*   **<font color='red'> Problema </font>** 
##### Detectar y clasificar la postura de una mascarilla sanitaria dentro de 3 clases.

<br>
<h6 align=center><b> ${\text{Tabla 1. Clases}}$ </b><h6>

|         ${\text{Class}}$         |      ${\text{Color Label}}$     |
|:--------------------------------:|:-------------------------------:|
|       ${\text{with mask}}$       |  $\color{green}{\text{green}}$  |
|     ${\text{without mask}}$      |    $\color{red}{\text{red}}$    |
| ${\text{mask weared incorrect}}$ | $\color{orange}{\text{orange}}$ |
<br>

*   **<font color='red'> Dataset </font>** 
##### El *dataset* está disponible en [<font color='purple'><i>**Drive**<i></font>](https://drive.google.com/drive/folders/12LAZUHCMIR65BISzW_e7Yozbq8tAKcAQ?usp=sharing), este dataset es construido con otros datasets de dominio público:
* [<font color='blue'><i>**Ver dataset 1**<i></font>](https://github.com/balajisrinivas/Face-Mask-Detection/tree/master/dataset)

* [<font color='blue'><i>**Ver dataset 2**<i></font>](https://github.com/cabani/MaskedFace-Net)
<br>

---
---

#####Desarrollado por: 
<h6 align=center> ${\text{Jhon Hader Fernández}}$ <h6>
<h6 align=center> ${\text{Diego Fernando Díaz}}$ <h6>
<h6 align=center> ${\text{Oscar Geovanny Baracaldo}}$ <h6>

#####<h6 align=center>{<i>jhon_fernandez, di-diego, obaracaldo</i>}@javeriana.edu.co<h6>
#####<h6 align=center>Pontificia Universidad Javeriana<h6>

<br>

## ***1. ENTORNO DE TRABAJO***

Para el desarrollo del sistema se requiere de algunos paquetes, módulos, librerías externas y dispositivos de ejecución.

### ***1.1.  INSTALACIÓN DE PAQUETES EXTERNOS*** 

In [ ]:
!pip install tensorflow

### ***1.2.  IMPORTAR PAQUETES Y LIBRERÍAS***




In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import os

### ***1.3.  VERIFICAR EL DISPOSITIVO GPU***

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found, please check the device')
print('Found GPU at: {}'.format(device_name))

## ***2. DATOS***

Se debe cargar los datos, realizar un pequeño analisis y pre-procesamiento.

### ***2.1.  OBTENER DATASET*** 
* **<font color='green'><i> 2.1.1. </i></font>** <br>
 El **dataset**  es cargado desde una carpeta de **<font color='magenta'> Google drive </font>** por ello se debe obtener la ruta de la carpeta, para ello se solicitará un permiso de acceso a **<font color='magenta'> Google drive </font>**

* **<font color='green'><i> 2.1.2. </i></font>** <br>
 Se cargan todas las imagenes del conjunto de datos con su respectiva etiqueta, todas las imagenes son ajustadas a un tamaño de **<font color='blue'> `(224, 224)` </font>**

In [ ]:
# 2.1.1
DIRECTORY  = '/content/drive/My Drive/Face Mask Detection/dataset'
CATEGORIES  = ['mask_weared_incorrect', 'with_mask', 'without_mask']

data = []
labels = []
i = 0

# 2.1.2
print("[INFO] loading images...")
for j, category in enumerate(CATEGORIES):
	path = os.path.join(DIRECTORY, category)
	for i, img in enumerate(os.listdir(path)):
		print('[INFO] Folder', j+1, 'of 3:', category)
		print('[INFO] Loading image', i+1, 'of', len(os.listdir(path)))
		img_path = os.path.join(path, img)
		image = load_img(img_path, target_size=(224, 224))
		image = img_to_array(image)
		image = preprocess_input(image)

		data.append(image)
		labels.append(category)
		clear_output(wait=True)
	
print("[INFO] Images was loaded successfully!")

### ***2.2.  BALANCE*** 
Se obtiene el balance del conjunto de datos, esto permitirá descartar sobre ajustes en el entrenamiento del modelo.

In [ ]:
balance = {}

for classes in CATEGORIES:
  balance[classes] = labels.count(classes)
  balance['% ' + classes] = (balance[classes] * 100.0) / len(labels)

tags = 'with_mask', 'without_mask', 'mask_weared_incorrect'
sizes = [balance['% with_mask'], balance['% without_mask'], balance['% mask_weared_incorrect']]
colors = ['yellowgreen', 'crimson', 'orangered']
explode = (0.05, 0.05, 0.05)

fig, ax1 = plt.subplots(figsize = (18,9)) 
ax1.pie(sizes, explode=explode, colors=colors, startangle=90, autopct='%.1f%%', pctdistance=0.85, shadow = True, textprops=dict(fontsize=17)) 
plt.title('Data balance', fontsize=18, color='red') 
ax1.legend(tags, title="Classes", loc='upper left', fontsize=12) 
plt.tight_layout() 
plt.show()

### ***2.3.  ONE HOT ENCONDING*** 

* **<font color='green'><i> 2.3.1. </i></font>** <br>
Integer encoding

* **<font color='green'><i> 2.3.2. </i></font>** <br>
One Hot encoding

In [ ]:
# 2.3.1
values = np.array(labels)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# 2.3.2
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)

data = np.array(data, dtype="float32")
labels = np.array(labels)

### ***2.4.  SEPARAR DATOS*** 

Se separa el conjunto de datos de la siguiente manera:

| ${\text{Data set}}$ | ${\text{%}}$  |
|:-------------------:|:-------------:|
|   ${\text{train}}$  | ${\text{80}}$ |
|   ${\text{test}}$   | ${\text{20}}$ |

<br>

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

### ***2.5.  AUGMENTATION*** 

Esto permite aumentar la cantidad de datos: al tener un conjunto de datos de imágenes se realizan transformaciones geométricas; trasladar, rotar, invertir, escalar, zoom positivo y negativo, etc. Siempre manteniendo la etiqueta original.

In [ ]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

## ***3. MODELO***

Se construye y entrena el modelo.

### ***3.1.  HIPER PARÁMETROS*** 

Se definen algunos hiper parámetros para el entrenamiento y desarrollo del modelo.

| ${\text{Hyper parameter}}$ | ${\text{Variable}}$ |
|:--------------------------:|:-------------------:|
|  ${\text{Learning rate}}$  |  ${\text{INIT_LR}}$ |
|   ${\text{Batch size}}$    |     ${\text{BS}}$   |
|     ${\text{Epochs}}$      |  ${\text{EPOCHS}}$  |

<br>

In [ ]:
INIT_LR = 1e-4          
BS = 32           
EPOCHS = 10

### ***3.2.  CONSTRUCCIÓN*** 

A continuación se observa la arquitectura del modelo con sus respectivos parámetros y confguraciones.

<br>

![arch.PNG](https://drive.google.com/uc?id=1ALPJrPRSN2MAFT9lnYl6ta7IjrogfjI2)

![arch1.PNG](https://drive.google.com/uc?id=1ynTxSdnK3veuN8KM9JviGho5br9_15VW)



In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
												input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# Crate model
model = Model(inputs=baseModel.input, outputs=headModel)

# Don't make fully conected the first layer
for layer in baseModel.layers:
	layer.trainable = False

# Compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
clear_output(wait=True)
print("[REPORT] model was compiled successfully!")

### ***3.3.  ENTRENAMIENTO*** 

Se entrena el modelo sobre una GPU.

In [ ]:
print("[INFO] training head...")

with tf.device('/device:GPU:0'):
  H = model.fit(
    aug.flow(trainX, trainY, batch_size=BS),
    steps_per_epoch=len(trainX) // BS,
    validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

print("[REPORT] model was trained successfully!")

### ***3.4.  EVALUACIÓN*** 

Se evalua el modelo implementado y se obtiene la precisión y la función de coste.

In [ ]:
print("[INFO] evaluating network...")
# make predictions on the testing set
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=CATEGORIES))

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

performance_path = os.path.join('/content/drive/My Drive/Face Mask Detection/model', 'evaluation.png')
plt.savefig(performance_path)

### ***3.4.  EXPORTAR MODELO*** 

Una vez el modelo es evaluado se exporta en formato **<font color='blue'> `.model` </font>** para ser usado como un modelo en cualquier otra aplicación.

In [ ]:
print("[INFO] saving mask detector model...")
model_path = os.path.join('/content/drive/My Drive/Face Mask Detection/model', 'mask_detector.model')
model.save(model_path, save_format="h5")
clear_output(wait=True)
print("[INFO] model was saved successfully!")